In [290]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime

# Code aggregation

In [ ]:
# 14 recipes per page with page from no figure then 2 to 833
#https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=10004&page=833

In [147]:
#list of links for the pages - start with 5pages to test before launching 833 pages...
# Check if pages numbering changing from 14 to 12 impacts the shape of the link
#Number of page does not impact the elements displayed ==> fix page number
def List_urls(NbPage):
    urls=[]
    for i in range(NbPage):
        url=f'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start={14*i}&page={2}'
        urls.append(url)
        i+=1
    return urls

page_urls=List_urls(833)    
print(len(page_urls))

833


In [148]:
#For one given page look for the list of recipes (links)
#14 recipes per pages (or 12 paging changing?) time sleep =0,5sec every requests)
def find_recipe_card_link(page_urls):
    recipes_of_all_pages=[]
    for url in page_urls:
        print('url page is',{url})
        soup=BeautifulSoup(r.get(url).content)
        # list of recipes url which are in the page
        recipes_of_all_pages.extend([f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')])
        time.sleep(0.5)
    return recipes_of_all_pages
        
recipes_of_all_pages=find_recipe_card_link(page_urls) 
# ten minutes to get 10404 links



In [149]:

print(len(recipes_of_all_pages))

10404


In [132]:

def Scrap_data(url):   
    
        
    soup=BeautifulSoup(r.get(url).content)
    
    # Recipe Name
    name=[i.text for i in soup.select('h1')]
       
    # Author name
    author_name=[i.text.strip('\n ') for i in soup.select('div.recipe-author__author>span')]
        
    #Rating
    rating= [i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]
        
    #Tags
    tags=[i.text for i in soup.select('ul.mrtn-tags-list>li.mrtn-tag>a')]
        
    # nb likes
    try:
        nblikes=soup.select('span.recipe-infos-users__value')[1].text
    except IndexError:
        nblikes=''
        
    #nb portions
    nbportions=[i.text.strip('\n') for i in soup.select('span.title-2.recipe-infos__quantity__value')]
        
    # difficulty
    difficulty=[i.text.strip('\n') for i in soup.select('div.recipe-infos__level')]
        
    #Total cooking time
    totaltime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__total-time ')]
        
    # preparation time
    preparationtime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__preparation')]
            
    # cooking only time
    cookingtime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__cooking')]

    # cost range
    costrange=[i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')]
        
    # add url
    link=url
        
    #list of ingredients
    loi=[i.text.strip('\n') for i in soup.select('li.recipe-ingredients__list__item')]
        
    #recipe (text)
    recipedescr=[i.text.strip('\n') for i in soup.select('ol.recipe-preparation__list')]
        
    dct={'Recipe_name':name,\
         'Author_name':author_name,\
         'Rating':rating,\
         'Tags':tags,\
         'Nb_likes':nblikes,\
         'Nb_portions':nbportions,\
         'Difficulty':difficulty,\
         'Total_cooking_time':totaltime,\
         'Preparation_time':preparationtime,\
         'Cooking_time':cookingtime,\
         'Cost_range':costrange,\
         'List_of_ingredients':loi,\
         'Recipe_description':recipedescr,\
         'Link':link}
    return dct




# Clean CSV file

In [422]:
data=pd.read_csv('/users/sandrine/Desktop/Prework/Projects_Ironhack/web_scraping/marmiton28082020.csv',index_col = 0)


In [423]:
data.head()

,Recipe_name,Author_name,Rating,Tags,Nb_likes,Nb_portions,Difficulty,Total_cooking_time,Preparation_time,Cooking_time,Cost_range,List_of_ingredients,Recipe_description,Link
0,['Ramequins fondants au chocolat'],['miss chocolat'],['4.8/5'],"['Végétarien', 'Gâteau au chocolat', 'Gâteau',...",189k,['4'],['très facile'],['Temps Total : 22 min\t\t\t\t\t\t\t\t'],['Préparation\xa0: 10 min\t\t\t\t\t\t\t\t\t\t'],['Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t12 m...,['bon marché'],['120\n\n\n\ng de chocolat noir + 8 carrés (5 ...,['\t\t\tFaites fondre dans une casserole le ch...,https://www.marmiton.org/recettes/recette_rame...
1,"[""'The' tarte au citron meringuée""]",['margot_16759605'],['4.7/5'],"['Végétarien', 'Tarte', 'Tarte au citron', 'Le...",162k,['6'],['très facile'],['Temps Total : 55 min\t\t\t\t\t\t\t\t'],['Préparation\xa0: 30 min\t\t\t\t\t\t\t\t\t\t'],['Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t25 m...,['bon marché'],"['250\n\n\n\ng de farine ', '125\n\n\n\ng de ...",['\t\t\tOn commence par la pâte sablée : mettr...,https://www.marmiton.org/recettes/recette_the-...
2,['Punch exotique'],['Marmiton_Recettes'],['3.2/5'],"['Sans gluten', 'Cocktail apéritif', 'Punch']",0,['20'],['facile'],[],[],[],['Coût moyen'],"[""4\n\n\n\nl de jus d'oranges "", '1\n\n\n\nve...","['\t\t\tRéalisez la recette ""Punch exotique"" d...",https://www.marmiton.org/recettes/recette_punc...
3,['Tiramisu (recette originale)'],['agathe_13870034'],['4.7/5'],"['Végétarien', 'Rapide', 'Tiramisu', ""Paradis ...",137k,['8'],['très facile'],['Temps Total : 15 min\t\t\t\t\t\t\t\t'],['Préparation\xa0: 15 min\t\t\t\t\t\t\t\t\t\t'],[],['bon marché'],"['3\n\n\n\n oeufs ', '100\n\n\n\ng de sucre r...","[""Séparer les blancs des jaunes d'oeufs.\t\t\n...",https://www.marmiton.org/recettes/recette_tira...
4,['Original American Cookies de Mike'],['nathan_15813882'],['4.7/5'],"['Végétarien', 'Rapide', 'Petits gâteaux', 'Gâ...",121k,['15'],['très facile'],['Temps Total : 15 min\t\t\t\t\t\t\t\t'],['Préparation\xa0: 5 min\t\t\t\t\t\t\t\t\t\t'],['Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t10 m...,['bon marché'],"['250\n\n\n\ng de farine ', '90\n\n\n\ng de s...","[""\t\t\tMélangez la farine, les sucres, le sel...",https://www.marmiton.org/recettes/recette_orig...


In [424]:
data.shape


(9601, 14)

In [425]:
data.drop_duplicates(inplace=True)

In [426]:
pd.Series(data.Recipe_name).value_counts()

["Confiture d'oignons"]                      3
['Fondant au chocolat']                      2
['Brownie']                                  2
['Risotto aux champignons']                  2
['Salade de pois chiches']                   2
                                            ..
["Tartare aux deux saumons et à l'aneth"]    1
['Quiche thon, tomates et mozzarella']       1
['Pasticcio (Grèce)']                        1
['Gâteau aux cerises']                       1
['Gâteau à la poudre de cacao']              1
Name: Recipe_name, Length: 8895, dtype: int64

In [427]:
data.shape

(8929, 14)

In [428]:
#remove square brackets generated by the list comprehension during the scrap (I chose to have them to avoid index errors and not using try except)
data=data.apply(lambda x: x.str.strip(('[')).str.strip(']'))


In [429]:
data.Cost_range=data.Cost_range.str.replace('bon marchÃ©','bon marché').str.replace('CoÃ»t moyen','Coût moyen')

In [430]:
data.Difficulty=data.Difficulty.str.replace('trÃ¨s facile','très facile')

In [431]:
pd.Series(data.Difficulty).value_counts()

'très facile'     5181
'facile'          3171
'Niveau moyen'     528
'difficile'         49
Name: Difficulty, dtype: int64

In [432]:
#cleaning cooking time columns
data.Preparation_time=data.Preparation_time.map(lambda x: re.sub(r'\\xa0:|\\t*','',x.replace('Préparation','')))
data.Cooking_time=data.Cooking_time.map(lambda x: re.sub(r'\\xa0:|\\t*|\\n*','',x.replace('Cuisson','')))
data.Cooking_time=data.Cooking_time.str.replace('nn','')
data.Total_cooking_time=data.Total_cooking_time.map(lambda x: re.sub(r'\\xa0:|\\t*|\\n*','',x.replace('Temps Total : ','')))

In [433]:
# cleaning List of Ingredients column
data.List_of_ingredients=data.List_of_ingredients.map(lambda x: re.sub(r'(\W&\S)|\\n*','',x))

In [434]:
# cleaning recipe text
data.Recipe_description=data.Recipe_description.map(lambda x: re.sub(r'\\n{2}|\\t{3}','',x))
data.Recipe_description.str.replace('\t\t','')

0       '\t\t\tFaites fondre dans une casserole le cho...
1       '\t\t\tOn commence par la pâte sablée : mettre...
2       '\t\t\tRéalisez la recette "Punch exotique" da...
3       "Séparer les blancs des jaunes d'oeufs.\t\t\n\...
4       "\t\t\tMélangez la farine, les sucres, le sel ...
                              ...                        
8983    "\t\t\tLaver les courgettes mais ne pas les ép...
8984    "\t\t\tEnlever la peau des mangues et coupez-l...
8985    '\t\t\tLaver les fraises, les équeuter. En met...
8986    "\t\t\tFaire cuire les pommes de terre à l'eau...
8987    "\t\t\tCouper la mangue en petits morceaux, et...
Name: Recipe_description, Length: 8929, dtype: object

In [435]:
#analysis:
On the most liked recipes what are the more common ingredients
Analysis of who write the recipes
How many times : chocolate, beurre, lait , allergenes

SyntaxError: invalid syntax (<ipython-input-435-217866f77b6c>, line 2)

In [382]:
data.head()

,Recipe_name,Author_name,Rating,Tags,Nb_likes,Nb_portions,Difficulty,Total_cooking_time,Preparation_time,Cooking_time,Cost_range,List_of_ingredients,Recipe_description,Link
0,'Ramequins fondants au chocolat','miss chocolat','4.8/5',"'Végétarien', 'Gâteau au chocolat', 'Gâteau', ...",189k,'4','très facile','22 min',' 10 min','12 min','bon marché',"'120g de chocolat noir + 8 carrés (5 g) ', '3 ...",'\t\t\tFaites fondre dans une casserole le cho...,https://www.marmiton.org/recettes/recette_rame...
1,"""'The' tarte au citron meringuée""",'margot_16759605','4.7/5',"'Végétarien', 'Tarte', 'Tarte au citron', 'Les...",162k,'6','très facile','55 min',' 30 min','25 min','bon marché',"'250g de farine ', '125g de beurre doux ', '...",'\t\t\tOn commence par la pâte sablée : mettre...,https://www.marmiton.org/recettes/recette_the-...
2,'Punch exotique','Marmiton_Recettes','3.2/5',"'Sans gluten', 'Cocktail apéritif', 'Punch'",0,'20','facile',,,,'Coût moyen',"""4l de jus d'oranges "", '1verre de sirop de s...","'\t\t\tRéalisez la recette ""Punch exotique"" da...",https://www.marmiton.org/recettes/recette_punc...
3,'Tiramisu (recette originale)','agathe_13870034','4.7/5',"'Végétarien', 'Rapide', 'Tiramisu', ""Paradis d...",137k,'8','très facile','15 min',' 15 min',,'bon marché',"'3 oeufs ', '100g de sucre roux ', '1sachet ...","""Séparer les blancs des jaunes d'oeufs.\t\t\n\...",https://www.marmiton.org/recettes/recette_tira...
4,'Original American Cookies de Mike','nathan_15813882','4.7/5',"'Végétarien', 'Rapide', 'Petits gâteaux', 'Gât...",121k,'15','très facile','15 min',' 5 min','10 min','bon marché',"'250g de farine ', '90g de sucre de canne rou...","""\t\t\tMélangez la farine, les sucres, le sel ...",https://www.marmiton.org/recettes/recette_orig...


In [436]:
data[['Recipe_name','List_of_ingredients','Link']]

,Recipe_name,List_of_ingredients,Link
0,'Ramequins fondants au chocolat',"'120g de chocolat noir + 8 carrés (5 g) ', '3 ...",https://www.marmiton.org/recettes/recette_rame...
1,"""'The' tarte au citron meringuée""","'250g de farine ', '125g de beurre doux ', '...",https://www.marmiton.org/recettes/recette_the-...
2,'Punch exotique',"""4l de jus d'oranges "", '1verre de sirop de s...",https://www.marmiton.org/recettes/recette_punc...
3,'Tiramisu (recette originale)',"'3 oeufs ', '100g de sucre roux ', '1sachet ...",https://www.marmiton.org/recettes/recette_tira...
4,'Original American Cookies de Mike',"'250g de farine ', '90g de sucre de canne rou...",https://www.marmiton.org/recettes/recette_orig...
...,...,...,...
8983,'Purée de courgettes simple',"'1kg de courgette ', '10cl de crème fraîche é...",https://www.marmiton.org/recettes/recette_pure...
8984,'Poulet aux mangues',"'1 poulet de 1,5 kg ', '3 mangues bien mûres '...",https://www.marmiton.org/recettes/recette_poul...
8985,'Crème à la fraise',"'500g de fraises ', '100g de sucre ', '1 cit...",https://www.marmiton.org/recettes/recette_crem...
8986,'Gratin de pommes de terre en béchamel',"'1.5kg de pomme de terre ', '100g de beurre ...",https://www.marmiton.org/recettes/recette_grat...


In [391]:
#give frequency of values in the whole column list of ingredients 
Ingredient_df=data.List_of_ingredients.str.lower().str.split(expand=True).stack().value_counts().to_dict()
#["','","'",'",',"'1","'2",'en',"'1cuillère",'100g",'ou',"'2cuillères","'150g",'"',"'50g","'250g",'(ou','500g",'et','pour','"1cuillère',"'3cuillères",'"2cuillères','le']
Ingredient_df


{"',": 56919,
 'de': 45995,
 'à': 9271,
 "'": 7864,
 '",': 7165,
 "'1": 6091,
 'soupe': 5480,
 'sucre': 4319,
 "'sel": 3487,
 "'poivre": 3441,
 "'2": 3383,
 'beurre': 3159,
 'farine': 2901,
 'en': 2880,
 'oeufs': 2746,
 "'1cuillère": 2677,
 'café': 2547,
 'crème': 2488,
 "'100g": 2303,
 'ou': 2228,
 "'3": 2013,
 'fraîche': 1961,
 'lait': 1936,
 'poudre': 1794,
 "'4": 1789,
 "d'olive": 1730,
 "d'huile": 1712,
 "'200g": 1587,
 "'2cuillères": 1511,
 'levure': 1379,
 "d'ail": 1360,
 "'150g": 1346,
 'blanc': 1290,
 'oignon': 1275,
 "'50g": 1261,
 'sel': 1222,
 "'250g": 1134,
 '"': 1074,
 'tomates': 1061,
 "'1pincée": 1031,
 "'1sachet": 1008,
 'chocolat': 1002,
 'râpé': 1002,
 'citron': 938,
 '(ou': 919,
 "'500g": 895,
 'pâte': 889,
 'liquide': 853,
 "d'eau": 848,
 'oignons': 819,
 'et': 814,
 "'125g": 791,
 "'20cl": 772,
 '"huile': 740,
 "d'oeuf": 731,
 'pour': 700,
 'bouillon': 695,
 'vin': 685,
 '"1cuillère': 683,
 'la': 668,
 'frais': 667,
 "'3cuillères": 665,
 'pommes': 662,
 'oeuf': 65

In [394]:
data[data.List_of_ingredients.str.lower().str.contains('soupe')]

,Recipe_name,Author_name,Rating,Tags,Nb_likes,Nb_portions,Difficulty,Total_cooking_time,Preparation_time,Cooking_time,Cost_range,List_of_ingredients,Recipe_description,Link
0,'Ramequins fondants au chocolat','miss chocolat','4.8/5',"'Végétarien', 'Gâteau au chocolat', 'Gâteau', ...",189k,'4','très facile','22 min',' 10 min','12 min','bon marché',"'120g de chocolat noir + 8 carrés (5 g) ', '3 ...",'\t\t\tFaites fondre dans une casserole le cho...,https://www.marmiton.org/recettes/recette_rame...
1,"""'The' tarte au citron meringuée""",'margot_16759605','4.7/5',"'Végétarien', 'Tarte', 'Tarte au citron', 'Les...",162k,'6','très facile','55 min',' 30 min','25 min','bon marché',"'250g de farine ', '125g de beurre doux ', '...",'\t\t\tOn commence par la pâte sablée : mettre...,https://www.marmiton.org/recettes/recette_the-...
10,'Les Timbales de Jeanne (saumon à la mousse de...,'sofia_15984013','4.8/5',"'Sans gluten', 'Micro-ondes'",78k,'4','très facile','18 min',' 15 min','3 min','bon marché',"'4tranches de saumon fumé ', '2 courgettes '...","'\t\t\tRincer, essorer et ciseler l\'aneth et ...",https://www.marmiton.org/recettes/recette_les-...
15,'Fondant au chocolat','Anonyme','4.6/5',"'Végétarien', 'Gâteau au chocolat', 'Gâteau', ...",109k,'8','très facile','35 min',' 15 min','20 min','bon marché',"'200g de chocolat à cuire ', '100g de beurre d...",'\t\t\tPréchauffer le four à 180°C (thermostat...,https://www.marmiton.org/recettes/recette_fond...
16,'Béchamel rapide et facile','Anonyme','4.8/5',"'Végétarien', 'Rapide', 'Béchamel', 'Micro-ondes'",82k,'4','très facile','7 min',' 5 min','2 min','bon marché',"'25cl de lait ', '20g de beurre ', '2cuillèr...","""\t\t\tDans un grand récipient, faire fondre l...",https://www.marmiton.org/recettes/recette_bech...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8964,'Soupe acidulée au potiron','Anonyme','4.3/5',"'Végétarien', 'Sans gluten', 'Entrée chaude', ...",883,'6','très facile','35 min',' 15 min','20 min','bon marché',"'1.5kg de potiron ', '1 pomme (boskoop ou rei...","""\t\t\tCouper le potiron et la pomme épluchés ...",https://www.marmiton.org/recettes/recette_soup...
8970,'Gâteau léger au chocolat sans beurre','margot_16343262','4.3/5',"'Végétarien', 'Gâteau', 'Gâteau au chocolat'",615,'6','très facile','1h05',' 20 min','45 min','bon marché',"'200g de chocolat pâtissier ', '3 oeufs ', '...",'\t\t\tPréchauffer le four à 150°C (thermostat...,https://www.marmiton.org/recettes/recette_gate...
8971,'Yaourt maison au citron','faust25','3.8/5',"'Végétarien', 'Sans gluten'",3.2k,'8','très facile','25 min',' 20 min','5 min','bon marché',"'1 yaourt nature ', '1 citron jaune ', '1l de...","""\t\t\tA préparer la veille.\t\t\n\n\t\t\tPort...",https://www.marmiton.org/recettes/recette_yaou...
8972,'BEGHRIR - Crèpes marocaines milles trous','sabrine1234','4.6/5','Végétarien',1.3k,'12','facile','1 h',' 30 min','30 min','bon marché',"'375g de semoule fine ', '125g de farine blan...","""\t\t\tVersez la levure sèche dans un récipien...",https://www.marmiton.org/recettes/recette_begh...


In [289]:
pd.Series(data.Cost_range).value_counts()

'bon marché'    6709
'Coût moyen'    2005
'assez cher'     215
Name: Cost_range, dtype: int64

In [333]:

data.Total_cooking_time


0       'Temps Total : 22 min\t\t\t\t\t\t\t\t'
1       'Temps Total : 55 min\t\t\t\t\t\t\t\t'
2                                             
3       'Temps Total : 15 min\t\t\t\t\t\t\t\t'
4       'Temps Total : 15 min\t\t\t\t\t\t\t\t'
                         ...                  
8983    'Temps Total : 25 min\t\t\t\t\t\t\t\t'
8984      'Temps Total : 1h10\t\t\t\t\t\t\t\t'
8985    'Temps Total : 10 min\t\t\t\t\t\t\t\t'
8986    'Temps Total : 50 min\t\t\t\t\t\t\t\t'
8987    'Temps Total : 35 min\t\t\t\t\t\t\t\t'
Name: Total_cooking_time, Length: 8929, dtype: object

In [395]:
pd.Series(data.Author_name).value_counts()

'Anonyme'                   1297
                             378
'Marmiton_Recettes'           86
'colorado'                    49
'Christophe_de_Marmiton'      32
                            ... 
'serge_167'                    1
'aurelie_2101'                 1
'claire_2253'                  1
'Chelly'                       1
'elena_16416145'               1
Name: Author_name, Length: 5379, dtype: int64

In [419]:
data.Nb_likes.to_dict()

{0: '189k',
 1: '162k',
 2: '0',
 3: '137k',
 4: '121k',
 5: '151k',
 6: '104k',
 7: '120k',
 8: '89k',
 9: '2.4k',
 10: '78k',
 11: '89k',
 12: '73k',
 13: '1.1k',
 14: '96k',
 15: '109k',
 16: '82k',
 17: '0',
 18: '105k',
 19: '51k',
 20: '96k',
 21: '97k',
 22: '0',
 23: '747',
 24: '64k',
 25: '94k',
 26: '66k',
 27: '1.8k',
 28: '104k',
 29: '77k',
 30: '61k',
 31: '64k',
 32: '110k',
 33: '79k',
 34: '81k',
 35: '47k',
 36: '73k',
 37: '374',
 38: '0',
 39: '91k',
 40: '68k',
 41: '0',
 42: '40k',
 43: '70k',
 44: '0',
 45: '67k',
 46: '87k',
 47: '77k',
 48: '38k',
 49: '36k',
 50: '36k',
 51: '169',
 52: '49k',
 53: '90k',
 54: '55k',
 55: '0',
 56: '72k',
 57: '88k',
 58: '44k',
 59: '47k',
 60: '40k',
 61: '59k',
 62: '0',
 63: '29k',
 64: '101k',
 65: '157',
 66: '41k',
 67: '32k',
 68: '70k',
 69: '20',
 70: '106k',
 71: '35k',
 72: '48k',
 73: '58k',
 74: '46k',
 75: '23k',
 76: '50k',
 77: '38k',
 78: '17k',
 79: '0',
 80: '47k',
 81: '67k',
 82: '41k',
 83: '85',
 84: '

In [411]:
data[data.Nb_likes=='999']

,Recipe_name,Author_name,Rating,Tags,Nb_likes,Nb_portions,Difficulty,Total_cooking_time,Preparation_time,Cooking_time,Cost_range,List_of_ingredients,Recipe_description,Link
6212,'Ragoût breton de saucisses bon et pas cher ','alya_16259053','4.9/5',"'Viande rôtie', 'Viande', 'Saucisses'",999,'4','très facile','55 min',' 10 min','45 min','bon marché',"'12 chipolata (ou autres aucisses) ', '1.2kg d...","""\t\t\tPour les novices : faire un roux blond ...",https://www.marmiton.org/recettes/recette_rago...
7320,'Pain de mie brioché','bjorn_15871022','4.3/5',"'Végétarien', 'Pain', 'Pain de mie'",999,'6','très facile','55 min',' 15 min','40 min','bon marché',"'450g de farine ordinaire type 45 ou 55 ', '25...","""\t\t\tCette préparation nécessite de reposer....",https://www.marmiton.org/recettes/recette_pain...
8242,'Sablés aux épices et amandes','Papa cuistot','4.7/5',"'Végétarien', 'Petits gâteaux', 'Gâteau', 'Sab...",999,'6','facile','30 min',' 20 min','10 min','Coût moyen',"'300g de farine ', '180g de beurre mou ', '16...","""\t\t\tTamisez la farine et les épices. \t\t\n...",https://www.marmiton.org/recettes/recette_sabl...
8263,'Mini gateau aux fraises Tagada','yaxa40','4.4/5','Végétarien',999,'10','facile','40 min',' 20 min','20 min','bon marché',"'25 fraises tagada® ® ', '80g de farine à gâte...","""\t\t\tPréchauffer votre four à 200°C (thermos...",https://www.marmiton.org/recettes/recette_mini...
